In [ ]:
"""
情感预测的练习。对于这个练习，
对于精度有影响的：
    1.增加训练集。5000->10000，精度提升10%。
    2.增加模型复杂度。simple->LSTM，更重要的是simple容易过拟合。
    3.stopword在此问题中不应该去除，会降低精度。主要原因应该是shouldn't之类可以判断情感的词被过滤掉了。
    4.增加epoch的次数。
减小过拟合的方法：
    1.加入dropout。
    2.降低模型复杂度。LSTM单元数减少。
    3.增加训练集。
"""

import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Activation, Dropout, Dense, LSTM, Bidirectional, Input 
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
from datetime import datetime
from tensorflow.keras import regularizers
import sys
sys.path.append("../common/")
import english_preprocess
import movie_review_helper

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch':[], 'epoch':[]}
        self.accuracy = {'batch':[], 'epoch':[]}
        self.val_loss = {'batch':[], 'epoch':[]}
        self.val_acc = {'batch':[], 'epoch':[]}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type, file_name):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.legend(loc="upper right")
        plt.savefig(file_name)       

def predict_single(review, model):
    review = tokenizer.texts_to_sequences(review)
    flat_list = []
    for sublist in review:
        for item in sublist:
            flat_list.append(item)
    flat_list = [flat_list]
    review = pad_sequences(flat_list, padding='post', maxlen=maxlen)
    print(model.predict(review))

tokenizer, embedding_mapping, X, Y, X_train, X_test, Y_train, Y_test = movie_review_helper.prepare_movie_review_for_task(10000, 100, 100)
maxlen = 100
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
def current_time():
    now = datetime.now()
    dt_string = now.strftime("%Y_%m_%d_%H_%M_%S")
    return dt_string

def epoch_performance(history, model_name):
    print("model name : {}".format(model_name))
    for i in history.epoch:
        print("epoch {}".format(i))
        print("train acc : {} validate acc : {}".format(history.history["acc"][i], history.history["val_acc"][i]))
        print("train loss : {} validate loss : {}".format(history.history["loss"][i], history.history["val_loss"][i]))

In [ ]:
def simple_model(model_name, dict_size, embedding_matrix, maxlen, X_train, X_test, Y_train, Y_test):
    model = Sequential()
    """
    All that the Embedding layer does is to map the integer inputs to the vectors found at the
    corresponding index in the embedding matrix,
    i.e. the sequence [1, 2] would be converted to [embeddings[1], embeddings[2]]
    这种模型overfitting比较严重。
    """
    model_info_file_prefix = "./model_info/{}_{}_".format(model_name, current_time())
    model_structure_file = model_info_file_prefix + "model_structure.png"
    embedding_layer = Embedding(dict_size, 100, weights=[embedding_matrix], input_length=maxlen, trainable=False)
    model.add(embedding_layer)
    model.add(Flatten())
    # 光通过这个解决不了overfitting的问题
    # model.add(Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    plot_model(model, to_file=model_structure_file , show_shapes=True)
    history_plot = LossHistory()
    history = model.fit(X_train, Y_train, batch_size=64, epochs=20, verbose=1, validation_split=0.2, callbacks=[history_plot])
    epoch_performance(history, model_name)
    history_plot.loss_plot('epoch', model_info_file_prefix+"epoch.png")
    score, accuracy = model.evaluate(X_test, Y_test, verbose=0)
    print("Test Score:", score)
    print("Test Accuracy:", accuracy)
    return model

In [ ]:
print("start training simple model")
model1 = simple_model("simple_model" ,vocab_size, embedding_mapping, maxlen, X_train, X_test, Y_train, Y_test)

In [ ]:
def LSTM_many_to_one_model(model_name, dict_size, embedding_matrix, maxlen, X_train, X_test, Y_train, Y_test):
    model_info_file_prefix = "./model_info/{}_{}_".format(model_name, current_time())
    model_structure_file = model_info_file_prefix + "model_structure.png"
    model = Sequential()
    embedding_layer = Embedding(dict_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
    model.add(embedding_layer)
    model.add(LSTM(64))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    plot_model(model, to_file=model_structure_file, show_shapes=True)
    if isinstance(Y, pd.core.series.Series):
        Y_train = Y_train.to_numpy()
    if isinstance(Y_test, pd.core.series.Series):
        Y_test = Y_test.to_numpy()
    history_plot = LossHistory()
    history = model.fit(X_train, Y_train, batch_size=64, epochs=20, verbose=1, validation_split=0.2, callbacks=[history_plot])
    history_plot.loss_plot('epoch', model_info_file_prefix+"epoch.png")
    epoch_performance(history, model_name)
    score, accuracy = model.evaluate(X_test, Y_test, verbose=0)
    print("Test Score:", score)
    print("Test Accuracy:", accuracy)
    return model

def LSTM_many_to_one_model_v2(model_name, dict_size, embedding_matrix, maxlen, X_train, X_test, Y_train, Y_test):
    """
    降低模型的overfit。
    """
    model_info_file_prefix = "./model_info/{}_{}_".format(model_name, current_time())
    model_structure_file = model_info_file_prefix + "model_structure.png"
    model = Sequential()
    embedding_layer = Embedding(dict_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
    model.add(embedding_layer)
    # 减少LSTM的个数
    model.add(LSTM(32))
    # 加入dropout
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    plot_model(model, to_file=model_structure_file, show_shapes=True)
    history_plot = LossHistory()
    history = model.fit(X_train, Y_train, batch_size=64, epochs=20, verbose=1, validation_split=0.2, callbacks=[history_plot])
    history_plot.loss_plot('epoch', model_info_file_prefix+"epoch.png")
    epoch_performance(history, model_name)
    score, accuracy = model.evaluate(X_test, Y_test, verbose=0)
    print("Test Score:", score)
    print("Test Accuracy:", accuracy)
    return model

In [ ]:
# print("start training LSTM v1 model")
# model3 = LSTM_many_to_one_model("LSTM_v1", vocab_size, embedding_mapping, maxlen, X_train, X_test, Y_train, Y_test)

# 能达到90%多
print("start training LSTM v2 model")
model4 = LSTM_many_to_one_model_v2("LSTM_v2", vocab_size, embedding_mapping, maxlen, X_train, X_test, Y_train, Y_test)

In [ ]:
def BI_LSTM_many_to_one_model(model_name, dict_size, embedding_matrix, maxlen, X_train, X_test, Y_train, Y_test):
    """
    降低模型的overfit。
    """
    model_info_file_prefix = "./model_info/{}_{}_".format(model_name, current_time())
    model_structure_file = model_info_file_prefix + "model_structure.png"
    model = Sequential()
    embedding_layer = Embedding(dict_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
    model.add(embedding_layer)
    model.add(Bidirectional(LSTM(32)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    plot_model(model, to_file=model_structure_file, show_shapes=True)
    if isinstance(Y, pd.core.series.Series):
        Y_train = Y_train.to_numpy()
    if isinstance(Y_test, pd.core.series.Series):
        Y_test = Y_test.to_numpy()
    history_plot = LossHistory()
    history = model.fit(X_train, Y_train, batch_size=64, epochs=20, verbose=1, validation_split=0.2, callbacks=[history_plot])
    history_plot.loss_plot('epoch', model_info_file_prefix+"epoch.png")
    epoch_performance(history, model_name)
    score, accuracy = model.evaluate(X_test, Y_test, verbose=0)
    print("Test Score:", score)
    print("Test Accuracy:", accuracy)
    return model

In [ ]:
print("start training BI_LSTM model")
model5 = BI_LSTM_many_to_one_model("BI_LSTM_v1", vocab_size, embedding_mapping, maxlen, X_train, X_test, Y_train, Y_test)

In [ ]:
def LSTM_many_to_one_model_v3(model_name, dict_size, embedding_matrix, X_train, X_test, Y_train, Y_test):
    """
    变长input，但是实际上通过不了。至少在tensorflow 1.15版本是不行的。
    无论是Sequential()还是function API。
    """
    model_info_file_prefix = "./model_info/{}_{}_".format(model_name, current_time())
    model_structure_file = model_info_file_prefix + "model_structure.png"
    
    embedding_input = Input(shape=(None,))
    embedding_output = Embedding(dict_size, 100, weights=[embedding_matrix], trainable=False, input_length=None)(embedding_input)
    LSTM_output = LSTM(32)(embedding_output)
    dropout_output = Dropout(0.5)(LSTM_output)
    dense_output = Dense(1, activation='sigmoid')(dropout_output)
    
    model = Model(embedding_input, dense_output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    
    plot_model(model, to_file=model_structure_file, show_shapes=True)
    if isinstance(Y, pd.core.series.Series):
        Y_train = Y_train.to_numpy()
    if isinstance(Y_test, pd.core.series.Series):
        Y_test = Y_test.to_numpy()
        
    history_plot = LossHistory()
    history = model.fit(X_train, Y_train, batch_size=64, epochs=20, verbose=1, validation_split=0.2, callbacks=[history_plot])
    history_plot.loss_plot('epoch', model_info_file_prefix+"epoch.png")
    epoch_performance(history, model_name)
    score, accuracy = model.evaluate(X_test, Y_test, verbose=0)
    print("Test Score:", score)
    print("Test Accuracy:", accuracy)
    return model

In [ ]:
# 实际上无法运行
# print("start training LSTM v3 model")
# model6 = LSTM_many_to_one_model_v3("LSTM_v3", vocab_size, embedding_mapping, X_train, X_test, Y_train, Y_test)

In [ ]:
def generate_model_callbacks(model_save_file):
    """
    加入early stopping
    降低learning rate
    """
    # save model callback
    checkpoint = ModelCheckpoint(model_save_file, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    # early stop callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')
    # reduce learning rate
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.000001, verbose=1)
    return [checkpoint, early_stopping, reduce_lr]
    
    
def LSTM_many_to_one_model_v4(model_name, dict_size, embedding_matrix, maxlen, X_train, X_test, Y_train, Y_test):
    """
    当前最高的val_acc是86%左右，想办法继续优化LSTM_many_to_one_model_v2
    1.尝试加入更多的dense+relu，两层dropout - done
    2.dense加入egularizer。 - done
    3.加入early stopping。 - done
    4.降低learning rate。- done
    5.两个LSTM?
    
    从实验的角度来看：
        dense加入egularizer没什么用。
        降低lr能小幅度提高。
        两层dropout也没什么用。
        early stopping只能节省时间。
    """
    model_info_file_prefix = "./model_info/{}_{}_".format(model_name, current_time())
    model_structure_file = model_info_file_prefix + "model_structure.png"
    model_save_file = model_info_file_prefix + "save_model"
    
    model = Sequential()
    embedding_layer = Embedding(dict_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
    model.add(embedding_layer)
    # 减少LSTM的个数
    model.add(LSTM(16))
    # 加入dropout
    model.add(Dropout(0.5))
    model.add(Dense(10,  kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l1(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    plot_model(model, to_file=model_structure_file, show_shapes=True)
    
    call_backs = generate_model_callbacks(model_save_file)
    history_plot = LossHistory()
    call_backs.append(history_plot)
    
    if isinstance(Y, pd.core.series.Series):
        Y_train = Y_train.to_numpy()
    if isinstance(Y_test, pd.core.series.Series):
        Y_test = Y_test.to_numpy()
    history = model.fit(X_train, Y_train, batch_size=64, epochs=20, verbose=1, validation_split=0.2, callbacks=call_backs)
    history_plot.loss_plot('epoch', model_info_file_prefix+"epoch.png")
    epoch_performance(history, model_name)
    score, accuracy = model.evaluate(X_test, Y_test, verbose=0)
    print("Test Score:", score)
    print("Test Accuracy:", accuracy)
    return model

In [ ]:
print("start training LSTM v4 model")
model7 = LSTM_many_to_one_model_v4("LSTM_v4", vocab_size, embedding_mapping, maxlen, X_train, X_test, Y_train, Y_test)